## Table of contents
   1. [**Estonian Universal Dependencies' EDT corpus**](#eesti_ud_edt_korpus)
      1. [**Converting UD corpus to Vabamorf format**](#eesti_ud_edt_konverteerimine)
      2. [**Creating and preparing the dataset from converted UD corpus**](#teisendatud_ud_andmestiku_loomine_ja_tootlemine)
      3. [**Model testing**](#mudeli_testimine)

In [1]:
import os
import csv
import re
import gc
import itertools
import json
import logging
import pkg_resources
import types
import pandas as pd
import sklearn as sk
import numpy as np
import estnltk
import torch
import simpletransformers
from simpletransformers.ner import NERModel, NERArgs
from est_ud_utils import load_ud_file_texts_with_corrections, load_ud_file_with_corrections
from est_ud_morph_conv import convert_ud_layer_to_reduced_morph_layer

e:\Anaconda3\envs\gpulocal\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:config.py:58: PyTorch version 2.4.0 available.


In [2]:
# Get locally imported modules from current notebook - https://stackoverflow.com/questions/40428931/package-for-listing-version-of-packages-used-in-a-jupyter-notebook - Alex P. Miller
def get_imports():
    
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
            
        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to add
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]
            
        yield name
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

estnltk==1.7.3
numpy==1.26.4
pandas==2.2.2
scikit-learn==1.5.1
simpletransformers==0.70.1
torch==2.4.0


<a id='eesti_ud_edt_korpus'></a>

### Estonian Universal Dependencies' EDT [corpus](https://github.com/UniversalDependencies/UD_Estonian-EDT)

<a id='eesti_ud_edt_konverteerimine'></a>

#### Converting UD corpus to Vabamorf format

[Convert Universal Dependencies' corpus to Vabamorf format notebook](https://github.com/estnltk/estnltk-model-training/blob/main/ud_morph_tools/amb_morph_reordering/01_convert_ud_corpus_to_vm.ipynb) was used to convert Estonian UD EDT treebank into Vabamorf format.

In [ ]:
def find_no_xpostag_rows():
    """
    Finds rows in the Estonian UD EDT treebank that contain rows where
    <code>xpostag == '_'</code>

    <i>In file <code>est_ud_utils.py</code> class <code>EstUDCorrectionsRewriter</code> has function <code>rewrite</code>, which has comment: \n
    #72: If <code>xpostag == '_'</code>, then add it based on upostag \n
    But not all xpostag conditions exist in the code as convertion throws an <code>AssertionError</code>.</i>
    """
    no_xpostag_regex = r"^\d+\t\S+\t\S+\t\S+\t_"
    conllu_dir = "UD_Estonian-EDT-r2.14"
    conllu_files = ["et_edt-ud-dev.conllu", "et_edt-ud-test.conllu", "et_edt-ud-train.conllu"]
    for c_file in conllu_files:
        print("\n", c_file, "\n")
        with open(file=os.path.join(conllu_dir, c_file), mode="r") as f:
            text = f.read()
            # Find all matches
            matches = re.findall(no_xpostag_regex, text, re.MULTILINE)

            # Print the matching rows
            for match in matches:
                print(match)

In [ ]:
ud_corpus_dir = "UD_Estonian-EDT-r2.14" # UD Corpus location
output_dir = 'UD_converted' # Output directory
# Create directory if it doesn't exist
if not os.path.isdir( output_dir ):
    os.makedirs(output_dir)
assert os.path.isdir( output_dir )

In [ ]:
# Load UD corpus' files as EstNLTK Text objects
loaded_texts  = []
ud_layer_name = 'ud_syntax'
for fname in os.listdir( ud_corpus_dir ):
    #if 'train' in fname:
    #    continue
    #if 'dev' in fname:
    #    continue
    #if 'test' in fname:
    #    continue
    if fname.endswith('.conllu'):
        fpath = os.path.join( ud_corpus_dir, fname )
        texts = load_ud_file_texts_with_corrections( fpath, ud_layer_name )
        for text in texts:
            text.meta['file'] = fname
            loaded_texts.append( text )

In [ ]:
# Convert UD's morphosyntactic annotations to Vabamorf-like annotations
for tid, text in enumerate(loaded_texts):
    convert_ud_layer_to_reduced_morph_layer( text, 'ud_syntax', 'ud_morph_reduced', add_layer=True )
    fname = text.meta['file'].replace('.conllu', '_'+('{:03d}'.format(tid))+'.json')
    fpath = os.path.join(output_dir, fname)
    estnltk.converters.text_to_json(text, file=fpath)

<a id='teisendatud_ud_andmestiku_loomine_ja_tootlemine'></a>

#### Creating and preparing the dataset from converted UD corpus

In [8]:
def create_df_ud_corpus(jsons, in_dir, csv_filename):
    """
    Creates a new dataset from converted the Estonian UD EDT <a href="https://github.com/UniversalDependencies/UD_Estonian-EDT">corpus</a>. \n
    For each <code>.json</code> file, the following info is gathered:
    <ul>
        <li><code>sentence_id</code> -- given for each sentence</li>
        <li><code>words</code> -- words gathered from text</li>
        <li><code>form</code> -- word form notation</li>
        <li><code>pos</code> -- part of speech</li>
        <li><code>file_prefix</code> -- metadata</li>
        <li><code>source</code> -- file name where the text is taken from</li>
    </ul>
    <a href="https://github.com/Filosoft/vabamorf/blob/e6d42371006710175f7ec328c98f90b122930555/doc/tagset.md">Tables of morphological categories</a> for more information about <code>form</code> and <code>pos</code>.

    Args:
        jsons (list): List of json files from which to read in the text
        in_dir (string): Directory containing list of files (<code>jsons</code>)
        csv_filename (string): CSV filename where to save the gathered text
    """
    tokens = list()
    sentence_id = 0
    fieldnames = ['sentence_id', 'words', 'form', 'pos', 'file_prefix', 'source']

    print("Beginning tokenization file by file. This can take a while.")
    for file_name in jsons:
        # print(f"Beginning to tokenize {file_name}")
        sentence_id = 0

        # Tokenization
        text = estnltk.converters.json_to_text(file=os.path.join(in_dir, file_name))
        file_prefix = text.meta.get('file_prefix')
        for sentence in text.sentences:
            sentence_analysis = sentence.ud_morph_reduced
            for text, form, pos in zip(sentence_analysis.text, sentence_analysis.form, sentence_analysis.pos):
                if text:
                    tokens.append((sentence_id, text, form[0], pos[0], file_prefix, file_name)) # In case of multiplicity, select the first or index 0
            sentence_id += 1
        # print(f"{file_name} tokenized")

    print("Tokenization completed successfully")
    print("Creating Pandas dataframe")
    df = pd.DataFrame(data=tokens, columns=fieldnames)
    df.to_csv(path_or_buf=csv_filename, index=False)
    print(f"Tokenized texts saved to {csv_filename}\n")

In [9]:
def clean_df(df, df_file_name):
    """Finishes dataframe by:
    <ul>
        <li>filling NaN values in columns <code>form</code> and <code>pos</code>with empty strings;</li>
        <li>removing NaN words.</li>
    </ul>

    Args:
        df (pandas.core.frame.DataFrame): Pandas dataframe to clean
        df_file_name (str): CSV file name from which dataframe was created
    """
    print("Assigning NaN values in columns form and pos with an empty string")
    # NaN values are assigned with an empty string
    df['form'] = df['form'].fillna('')
    df['pos'] = df['pos'].fillna('')
    print("Removing NaN words")
    # Removing NaN words
    df.dropna(subset=['words'], inplace=True)
    if df_file_name:
        df.to_csv(path_or_buf=df_file_name, index=False)
        print(f"Modified dataframe saved to {df_file_name}")
    else:
        print("Dataframe cleaned")

In [10]:
# New 'labels' column
def create_labels_column(df):
    """
    Creates a new column <code>labels</code> concatenating the values of columns <code>pos</code> (part of speech) and <code>form</code> (word form notation)

    Args:
        df (pandas.core.frame.DataFrame): Pandas dataframe to create a new column
    """
    df['labels'] = df.apply(lambda row: str(row['form']) + '_' + str(row['pos']) if row['form'] and row['pos'] else str(row['form']) or str(row['pos']), axis=1)
    print("Column 'labels' created")

In [58]:
ud_dir = "UD_converted"
jsons = os.listdir(ud_dir)
print(jsons)

['et_edt-ud-dev_000.json', 'et_edt-ud-dev_001.json', 'et_edt-ud-dev_002.json', 'et_edt-ud-dev_003.json', 'et_edt-ud-dev_004.json', 'et_edt-ud-dev_005.json', 'et_edt-ud-dev_006.json', 'et_edt-ud-dev_007.json', 'et_edt-ud-dev_008.json', 'et_edt-ud-test_009.json', 'et_edt-ud-test_010.json', 'et_edt-ud-test_011.json', 'et_edt-ud-test_012.json', 'et_edt-ud-test_013.json', 'et_edt-ud-test_014.json', 'et_edt-ud-train_015.json', 'et_edt-ud-train_016.json', 'et_edt-ud-train_017.json', 'et_edt-ud-train_018.json', 'et_edt-ud-train_019.json', 'et_edt-ud-train_020.json', 'et_edt-ud-train_021.json', 'et_edt-ud-train_022.json', 'et_edt-ud-train_023.json', 'et_edt-ud-train_024.json', 'et_edt-ud-train_025.json', 'et_edt-ud-train_026.json', 'et_edt-ud-train_027.json', 'et_edt-ud-train_028.json', 'et_edt-ud-train_029.json', 'et_edt-ud-train_030.json', 'et_edt-ud-train_031.json', 'et_edt-ud-train_032.json', 'et_edt-ud-train_033.json', 'et_edt-ud-train_034.json', 'et_edt-ud-train_035.json', 'et_edt-ud-trai

In [59]:
# if not os.path.exists('.\\ud_andmestik.csv'):
create_df_ud_corpus(jsons, ud_dir, 'ud_andmestik.csv')

Beginning tokenization file by file. This can take a while.
Tokenization completed successfully
Creating Pandas dataframe
Tokenized texts saved to ud_andmestik.csv



In [11]:
csv_ud_file = "ud_andmestik.csv"
df_ud = pd.read_csv(csv_ud_file, keep_default_na=False)

In [61]:
clean_df(df_ud, csv_ud_file)

Assigning NaN values in columns form and pos with an empty string
Removing NaN words
Modified dataframe saved to ud_andmestik.csv


In [12]:
create_labels_column(df_ud)
display(df_ud.head(5))
print(df_ud.shape)

Column 'labels' created


,sentence_id,words,form,pos,file_prefix,source,labels
0,0,Aga,,J,aja_ee199920,et_edt-ud-dev_000.json,J
1,0,mulle,sg all,P,aja_ee199920,et_edt-ud-dev_000.json,sg all_P
2,0,tundub,b,V,aja_ee199920,et_edt-ud-dev_000.json,b_V
3,0,",",,Z,aja_ee199920,et_edt-ud-dev_000.json,Z
4,0,et,,J,aja_ee199920,et_edt-ud-dev_000.json,J


(437826, 7)


<a id='mudeli_testimine'></a>

#### Model testing with UD corpus

Extracting test set

In [61]:
test_df_ud = df_ud[df_ud['source'].str.contains('ud-test')]
print(test_df_ud.shape)


(48489, 7)


Removing unnecessary columns for the model<!-- Mudelile ebavajalike veergude eemaldamine -->

In [62]:
test_df_ud = test_df_ud.drop(labels=['file_prefix', 'source'], axis=1)

Reading in unique labels that the model is trained with

In [63]:
# Reading in unique labels
with open("unique_labels.json", 'r') as f:
    unique_labels = json.load(f)

Labels unknown to Vabamorf and replacing them with appropriate known unique labels

In [64]:
def unknown_labels(unique_labels_list, data):
    """Finds labels that are not present in the unique labels list.

    Args:
        unique_labels_list (list): list of unique labels (obtained from reading <code>unique_labels.json</code> file)
        data (pandas.core.frame.DataFrame): data to check for labels
    """
    df_unique_labels = pd.DataFrame(unique_labels_list, columns=['labels'])
    unique_labels_series = df_unique_labels['labels']
    df_labels = data['labels']#.drop_duplicates()
    labels_not_in_unique = df_labels[~df_labels.isin(unique_labels_series)]

    print("Labels in data that are not in unique labels list:")
    print(labels_not_in_unique)
    print("Unique:")
    print(labels_not_in_unique.unique())
    return labels_not_in_unique

In [65]:
labels_not_in_unique = unknown_labels(unique_labels, test_df_ud)

Labels in data that are not in unique labels list:
46373    T
46374    T
46375    T
48036    T
48037    T
        ..
85108    T
85682    T
85683    T
85941    T
85942    T
Name: labels, Length: 85, dtype: object
Unique:
['T' 'sg g_place']


In [66]:
# print(labels_not_in_unique[20:40])

In [67]:
# display(df_ud[46368:46378])
# display(df_ud[69275:69281])

In [68]:
# Replace unknown pos 'T' with '?'
test_df_ud['labels'] = test_df_ud['labels'].apply(lambda x: '?' if x == 'T' else x)
# Replace unknown pos 'place' with 'S'
test_df_ud['labels'] = test_df_ud['labels'].apply(lambda x: 'sg g_S' if x == 'sg g_place' else x)

In [69]:
labels_not_in_unique = unknown_labels(unique_labels, test_df_ud)

Labels in data that are not in unique labels list:
Series([], Name: labels, dtype: object)
Unique:
[]


Initializing the model<!-- Mudeli ülesehitamine -->

In [70]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Configurations
model_args = NERArgs()
model_args.train_batch_size = 8
model_args.evaluate_during_training = False
model_args.learning_rate = 5e-5
model_args.num_train_epochs = 10
model_args.use_early_stopping = True
model_args.use_cuda = torch.cuda.is_available()  # Use GPU if available
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False # Takes a lot of storage space
model_args.save_steps = -1
model_args.overwrite_output_dir = True
model_args.cache_dir = 'NER_mudel/cache'
model_args.best_model_dir = 'NER_mudel/best_model'
model_args.output_dir = 'NER_mudel'
model_args.use_multiprocessing = False

# Initialization
model = NERModel("camembert", "NER_mudel", args=model_args, labels=unique_labels)

e:\Anaconda3\envs\gpulocal\Lib\site-packages\simpletransformers\ner\ner_model.py:433: UserWarning: use_multiprocessing automatically disabled as CamemBERT fails when using multiprocessing for feature conversion.
  warnings.warn(


Evaluating the model<!-- Mudeli hindamine -->

In [71]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(test_df_ud)
print(result)

INFO:ner_model.py:1884:  Converting to features started.


Running Evaluation:   0%|          | 0/12 [00:00<?, ?it/s]e:\Anaconda3\envs\gpulocal\Lib\site-packages\simpletransformers\ner\ner_model.py:1303: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
Running Evaluation: 100%|██████████| 12/12 [00:02<00:00,  5.04it/s]
e:\Anaconda3\envs\gpulocal\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: D seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
e:\Anaconda3\envs\gpulocal\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: pl p_A seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
e:\Anaconda3\envs\gpulocal\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: pl p_S seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
e:\Anaconda3\envs\gpulocal\Lib\site-packages\seqeval\metrics\sequence_lab

INFO:ner_model.py:1253: {'eval_loss': 0.9056411236524582, 'precision': 0.8831920868852051, 'recall': 0.9080618372301009, 'f1_score': 0.8954543163836114}
{'eval_loss': 0.9056411236524582, 'precision': 0.8831920868852051, 'recall': 0.9080618372301009, 'f1_score': 0.8954543163836114}


| eval_loss          | precision          | recall             | f1_score           |
|--------------------|--------------------|--------------------|--------------------|
| 0.9056411236524582 | 0.8831920868852051 | 0.9080618372301009 | 0.8954543163836114 |